In [76]:
import sys
import os
from os.path import split, join
base_dir = split(os.getcwd())[0]
if base_dir not in sys.path:
    sys.path.append(base_dir)
from utilities.basic import list_subdir
from utilities import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import json
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')


In [77]:
data_dir = join(base_dir, 'data')
data_long_dir = join(data_dir, 't1_longitudinal')

In [79]:
pred_df = pd.read_feather(join(data_long_dir, 'results', 'volume_prediction_results.feather'))
pred_df.InstanceCreationDate = pd.to_datetime(pred_df.InstanceCreationDate, format='%Y-%m-%d')
with open(join(data_long_dir, 'final', 'affected_brain_regions_dic.pkl'), 'rb') as f:
    affected_br_dic = pickle.load(f)
brain_regions_ls = [item for sublist in list(affected_br_dic.values()) for item in sublist]
with open(join(data_long_dir, 'final', 'long_sids_pairs.pkl'), 'rb') as f:
    long_sids_pairs = pickle.load(f)
with open(join(data_long_dir, 'final', 'cc_dic_ps_optmatch_const10.pkl'), 'rb') as f:
    cc_dic_ps10 = pickle.load(f)

In [91]:
df = get_p_value(brain_regions_ls, long_sids_pairs, cc_dic_ps10, pred_df)

In [92]:
df

In [90]:
def get_p_value(brain_regions_ls, sids, case_control_dic, pred_df):
    all_patients = [item for sublist in list(case_control_dic.values()) for item in sublist]\
        + list(case_control_dic.keys())
    pred_df = pred_df[pred_df.PatientID.isin(all_patients)]
    pred_df = pred_df[pred_df.SeriesInstanceUID.isin(sids)]
    pred_df['Date_diff'] = pred_df.groupby('PatientID')['InstanceCreationDate'].apply(lambda x: x.dt.date - x.min().date())# compute difference in dates
    df_pat = pd.DataFrame(df.groupby('PatientID').Date_diff.max()).reset_index()
    df_test = pred_df.sort_values(['Date_diff'],ascending=True).groupby('PatientID')
    return df_test